In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [2]:
class CricketState(TypedDict):
    balls: int
    runs: int
    fours: int 
    sixes: int 
    
    sr: float 
    bpb: float 
    boundary_percent: float 
    summary: str

### Writing core functions

In [23]:
def calculate_sr(state: CricketState) -> CricketState:
    sr = (state['runs']/state['balls'])*100
        
    return {'sr': sr}

In [24]:
def calculate_bpb(state: CricketState) -> CricketState:
    bpb = state['balls']/(state['fours'] + state['sixes'])
    
    return {'bpb': bpb}

In [25]:
def calculate_boundary_percent(state: CricketState) -> CricketState:
    boundary_percent = (((state['fours'] * 4) + (state['sixes'] * 6))/state['runs'])*100
    
    return {'boundary_percent': boundary_percent}

In [26]:
def summary(state: CricketState) -> CricketState:
    summary = f"""
Strike Rate - {state['sr']} \n
Balls per boundary - {state['bpb']} \n
Boundary percent - {state['boundary_percent']}"""

    return {'summary': summary}

### Adding Nodes and Edges

In [27]:
graph = StateGraph(CricketState)

graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary_percent', calculate_boundary_percent)
graph.add_node('summary', summary)

graph.add_edge(START, 'calculate_sr') 
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percent')
graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary_percent', 'summary')
graph.add_edge('summary', END)

workflow = graph.compile()

In [30]:
initial_state = {
    'balls': 50,
    'runs': 100,
    'fours': 6,
    'sixes': 4
}

result = workflow.invoke(initial_state)

In [31]:
print(result['summary'])


Strike Rate - 200.0 

Balls per boundary - 5.0 

Boundary percent - 48.0
